データの前処理

1. ファイルを指定してデータを読み込む
2. 回答ありのカラムを追加する
3. タイムスタンプをdatetimeオブジェクトにする
4. 国を大州と地域に分割する　←　カラムを追加する
5. カラムをcategoryにする
6. コメントのある／なしフラグを追加する　←　カラムを追加する
7. （他にあれば追加する）
8. ファイルをCSVで書き出す

In [ ]:
from pathlib import Path
import pandas as pd
import altair as alt
import textblob as tb
import titanite as ti

print(f"Altair {alt.__version__}")
print(f"Pandas {pd.__version__}")
print(f"Textblob {tb.__version__}")
print(f"Titanite {ti.__version__}")

設定ファイルを読み込む

In [ ]:
f_cfg = "../sandbox/config.toml"
config = ti.Config(load_from=f_cfg)
config.load()
# config.questions
# config.choices
# dir(config)

データを読み込む

- アンケート結果の生データを読み込む
- 生データはGoogleスプレッドシートをCSV形式で ``data/raw_data`` にダウンロードした
- リポジトリには追加しない（``.gitignore``してある）

In [ ]:
read_from = Path("../data/raw_data/20230722_icrc2023_diversity_presurvey_answers.csv")
data = pd.read_csv(read_from, skiprows=1)
data["response"] = 1

In [ ]:
# data.info()

日付のカラムを``datetime``オブジェクトに変換する

In [ ]:
data["timestamp"] = pd.to_datetime(data["timestamp"])

カラムの値を置換して整理する

- ``q03``（勤務地）と``q04``（出身地）のデータの表記を揃える
  - ``Oceania`` → ``Oceania / Oceania``
  - ``Prefer not to answer`` → ``Prefer not to answer / Prefer not to answer``
- ``q14``の回答の表記を揃える
  - ``No Interest`` → ``No interest``

In [ ]:
data["q03"] = data["q03"].replace(
    {
        "Prefer not to answer": "Prefer not to answer / Prefer not to answer",
        "Oceania": "Oceania / Oceania",
    }
)
data["q04"] = data["q04"].replace(
    {
        "Prefer not to answer": "Prefer not to answer / Prefer not to answer",
        "Oceania": "Oceania / Oceania",
    }
)
data["q14"] = data["q14"].replace({"No Interest": "No interest"})

- ``q03``と``q04``の回答を``/``で分割する
- ``大州 / 地域``という選択肢になっている
- ``_regional`` / ``_subregional`` の添字をカラム名に追加する

In [ ]:
_q3 = data["q03"].str.split("/", expand=True)
_q3[0] = _q3[0].str.strip()
_q3[1] = _q3[1].str.strip()
_q3 = _q3.rename(columns={0: "q03_regional", 1: "q03_subregional"})

_q4 = data["q04"].str.split("/", expand=True)
_q4[0] = _q4[0].str.strip()
_q4[1] = _q4[1].str.strip()
_q4 = _q4.rename(columns={0: "q04_regional", 1: "q04_subregional"})

元のデータに結合する

- 列方向に追加する（``axis=1``）
- これは一度だけ実施する
  - とくにガードをかけてないので、気をつける

In [ ]:
data = pd.concat([data, _q3], axis=1)
data = pd.concat([data, _q4], axis=1)
# data

In [ ]:
data.columns

``q10``と ``q13``の回答をビン分割する

- それぞれ整数値の回答だが、集計しやすいようにビン分割する
  - ``q10`` : ``0 - 24``の時間数。10時間以上は``10+``にまとめる
  - ``q13`` : ``0 - 100``のパーセンテージ。5%ずつ区切る
- ビン分割したカラムは``_binned``をつけて新しく作成する  

In [ ]:
data["q10"].unique()

In [ ]:
d = pd.cut(
    data["q10"],
    [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 25],
    right=False,
    labels=["NA", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10+"],
)

alt.Chart(d.reset_index()).mark_bar().encode(x="q10", y="count()")

In [ ]:
# data["q13"].value_counts()
# d = pd.cut(data["q13"], [-1, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], right=False, labels=["NA", "0", "10", "20", "30", "40", "50", "60", "70", "80", "90"])
d = pd.cut(
    data["q13"],
    [
        -1,
        0,
        5,
        10,
        15,
        20,
        25,
        30,
        35,
        40,
        45,
        50,
        55,
        60,
        65,
        70,
        75,
        80,
        85,
        90,
        95,
        100,
        105,
    ],
    right=False,
    labels=[
        "NA",
        "0%",
        "5%",
        "10%",
        "15%",
        "20%",
        "25%",
        "30%",
        "35%",
        "40%",
        "45%",
        "50%",
        "55%",
        "60%",
        "65%",
        "70%",
        "75%",
        "80%",
        "85%",
        "90%",
        "95%",
        "100%",
    ],
)
d.reset_index()
alt.Chart(d.reset_index()).mark_bar().encode(x="q13", y="count()")

カテゴリー型に変換する

- ``ti.config.categorical()``はdeprecatedにした
- ``Config.categories``でメンバー変数としてアクセスできるようにした

In [ ]:
# category = config.categorical()
category = config.categories

In [ ]:
# data["q01"] = data["q01"].astype(category["age"])
# data["q02"] = data["q02"].astype(category["gender"])
# data["q03"] = data["q03"].astype(category["geoscheme"])
# data["q03_regional"] = data["q03_regional"].astype(category["regional"])
# data["q03_subregional"] = data["q03_subregional"].astype(category["subregional"])
# data["q04"] = data["q04"].astype(category["geoscheme"])
# data["q04_regional"] = data["q04_regional"].astype(category["regional"])
# data["q04_subregional"] = data["q04_subregional"].astype(category["subregional"])
# data["q05"] = data["q05"].astype(category["job_title"])
# data["q06"] = data["q06"].astype(category["research_group"])
# data["q07"] = data["q07"].astype(category["research_field"]).fillna("Others")
# data["q08"] = data["q08"].astype(category["research_years"])
# data["q09"] = data["q09"].astype(category["yes_no"])
# # data["q10"]
# data["q11"] = data["q11"].astype(category["yes_no"])
# data["q12_genderbalance"] = data["q12_genderbalance"].astype(category["good_poor"])
# data["q12_diversity"] = data["q12_diversity"].astype(category["good_poor"])
# data["q12_equity"] = data["q12_equity"].astype(category["good_poor"])
# data["q12_inclusion"] = data["q12_inclusion"].astype(category["good_poor"])
# # data["q13"]
# data["q14"] = data["q14"].astype(category["good_poor"])
# # data["q15"]
# # data["q16"]
# data["q17_genderbalance"] = data["q17_genderbalance"].astype(category["agree_disagree"])
# data["q17_diversity"] = data["q17_diversity"].astype(category["agree_disagree"])
# data["q17_equity"] = data["q17_equity"].astype(category["agree_disagree"])
# data["q17_inclusion"] = data["q17_inclusion"].astype(category["agree_disagree"])
# # data["q18"]
# data["q19"] = data["q19"].astype(category["school"])
# # data["q20"]
# # data["q21"]
# # data["q22"]

In [ ]:
data.info()

上の内容を辞書型を使って見やすくした

In [ ]:
convert_to = {
    "q01": "age",
    "q02": "gender",
    "q03": "geoscheme",
    "q03_regional": "regional",
    "q03_subregional": "subregional",
    "q04": "geoscheme",
    "q04_regional": "regional",
    "q04_subregional": "subregional",
    "q05": "job_title",
    "q06": "research_group",
    "q07": "research_field",
    "q08": "research_years",
    "q09": "yes_no",
    "q11": "yes_no",
    "q12_genderbalance": "good_poor",
    "q12_diversity": "good_poor",
    "q12_equity": "good_poor",
    "q12_inclusion": "good_poor",
    "q14": "good_poor",
    "q17_genderbalance": "agree_disagree",
    "q17_diversity": "agree_disagree",
    "q17_equity": "agree_disagree",
    "q17_inclusion": "agree_disagree",
    "q19": "school",
}

categories = config.categories
for k, v in convert_to.items():
    c = categories.get(v, "category")
    data[k] = data[k].astype(c)

# 別途追加
data["q07"] = data["q07"].fillna("Others")

In [ ]:
# data.info()

for k in convert_to.keys():
    print(data[k].unique())

自由記述の感情分析をする

- 対象となるカラム: ``q15`` / ``q16`` / ``q18`` / ``q20`` / ``q21`` / ``q22``
- 内容を分析して、ポジティブ、ニュートラル、ネガティブに分類する
  - 文脈を判断するわけではないのが注意点 : https://qiita.com/K_Nemoto/items/28a817d57706d536d625
```python
$ python -m textblob.download_corpora
[nltk_data] Downloading package brown to /Users/shotakaha/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /Users/shotakaha/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shotakaha/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shotakaha/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/shotakaha/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/shotakaha/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.
``````

In [ ]:
from textblob import TextBlob
import numpy as np


def sentiment_polarity(text):
    try:
        blob = TextBlob(text)
        return blob.sentiment.polarity
    except TypeError as e:
        # print(e)
        return np.nan


def sentiment_subjectivity(text):
    try:
        blob = TextBlob(text)
        return blob.sentiment.subjectivity
    except TypeError as e:
        # print(e)
        return np.nan


def translation(text):
    try:
        blob = TextBlob(text)
        return blob.translate(from_lang="en", to="ja")
    except (TypeError, AttributeError) as e:
        print(e)
        return np.nan